In [1]:
# ==========================================================
# ATOMIC SHUTTLE — Multi-Screen UI with Dedicated Loading Page
# (Landing → Overview → Controls → Loading → Results → QR).
# Background image fills the entire app window on every screen.
# ==========================================================

import os, io, base64, urllib.request, time
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.patheffects as pe
from matplotlib.ticker import FixedLocator, FixedFormatter
from scipy.interpolate import splrep, BSpline
import ipywidgets as widgets
from IPython.display import display, clear_output, Image
import qrcode
from scipy.sparse import diags
from scipy.sparse.linalg import eigsh
from PIL import Image as PILImage  # avoid clash with IPython.display.Image
import tempfile
# one temp folder per process/session
TMPDIR = os.path.join(tempfile.gettempdir(), f"shuttleit-{os.getpid()}")
os.makedirs(TMPDIR, exist_ok=True)

# --- Stack polyfill for ipywidgets < 8 (NanoHub likely has v7.x)
try:
    _ = widgets.Stack  # ipywidgets 8+
    Stack = widgets.Stack
except AttributeError:
    class Stack(widgets.VBox):
        def __init__(self, children=(), **kwargs):
            super().__init__(children=children, **kwargs)
            self._selected_index = 0
            self._apply_visibility()

        @property
        def selected_index(self):
            return self._selected_index

        @selected_index.setter
        def selected_index(self, idx):
            self._selected_index = int(idx)
            self._apply_visibility()

        def _apply_visibility(self):
            # show only the selected child; hide the rest
            for i, ch in enumerate(self.children):
                ch.layout.display = '' if i == self._selected_index else 'none'

# =========================
# Paper-quality Matplotlib
# =========================
mpl.rcParams.update({
    "font.family": "DejaVu Sans",
    "mathtext.fontset": "stix",
    "font.size": 12,
    "axes.labelsize": 14,
    "axes.titlesize": 16,
    "xtick.labelsize": 12,
    "ytick.labelsize": 12,
    "axes.linewidth": 1.2,
    "lines.linewidth": 2.2,
    "figure.dpi": 240,       # ↑ crisper inline
    "savefig.dpi": 300,      # ↑ print/export
    "figure.constrained_layout.use": True,
    "pdf.fonttype": 42,      # embed TrueType
    "ps.fonttype": 42,
    "patch.antialiased": True,
    "lines.antialiased": True,
})

# ---------------------------
# Asset management
# ---------------------------
def data_uri(path):
    with open(path, "rb") as f:
        b = f.read()
    if path.lower().endswith((".jpg", ".jpeg")):
        mime = "image/jpeg"
    elif path.lower().endswith(".png"):
        mime = "image/png"
    elif path.lower().endswith(".svg"):
        mime = "image/svg+xml"
    else:
        mime = "application/octet-stream"
    return f"data:{mime};base64,{base64.b64encode(b).decode()}"

# https://www.svgator.com/blog/animated-svg-backgrounds-examples/
BG_URI   = data_uri("space-background.svg")
ATOM_URI = data_uri("atom.svg")

# ---------------------------
# Global state used by backend (keep names!)
# ---------------------------
global score
score = 0
is_running = False  # run guard to prevent double-runs

# ---------------------------
# Scoped CSS (applies ONLY inside .atomic-app container)
#   - Make plot containers expand (no scrollbars).
#   - Ensure Output widgets don't clip/scroll.
# ---------------------------
theme_css = f"""
<style>
.atomic-app {{
  position: relative;
  background-image: url('{BG_URI}');
  background-size: cover;
  background-repeat: no-repeat;
  background-position: center;
  background-color: #0a0d1a;

  border-radius: 18px;
  padding: 16px;
  box-shadow: 0 8px 24px rgba(0,0,0,0.35), inset 0 0 140px rgba(0,0,0,0.35);
  color: #eef3ff;
  font-family: "IBM Plex Mono", ui-monospace, SFMono-Regular, Menlo, Monaco, Consolas, "Liberation Mono", "Courier New", monospace;
  border: 1px solid rgba(0,230,255,0.25);

  min-height: 720px;
  width: 100%;
  box-sizing: border-box;
}}

.atomic-app .screen {{
  min-height: 688px;
  display: flex;
  flex-direction: column;
  gap: 10px;
}}

.atomic-app .panel {{
  background: rgba(12,12,30,0.78);
  border: 1px solid rgba(0,230,255,0.22);
  border-radius: 14px;
  box-shadow: 0 0 35px rgba(0,230,255,0.18), inset 0 0 16px rgba(255,255,255,0.04);
  backdrop-filter: blur(4px);
  padding: 14px;
  color: #ffffff;
}}

.atomic-app .panel-strong {{
  background: rgba(12,12,30,0.92);
  color: #ffffff;
}}

.atomic-app .panel :is(p, li, div, span, em, strong, b, code, h1, h2, h3, h4) {{
  color: inherit;
}}

.atomic-app h1.qtitle {{
  margin: 6px 0 0;
  text-align: center;
  font-weight: 700;
  letter-spacing: 1px;
  font-size: 2.6rem;
  color: #00e6ff;
  text-shadow: 0 0 22px rgba(0,230,255,0.6), 0 0 4px rgba(0,230,255,0.9);
}}

.atomic-app .subtitle {{
  margin: 4px auto 12px;
  text-align: center;
  max-width: 900px;
  color: #dfe7ff;
  background: linear-gradient(90deg, transparent, rgba(12,12,30,0.78), transparent);
  padding: 6px 12px;
  border-radius: 8px;
  font-size: 0.95rem;
}}

.atomic-app .q-btn {{
  border-radius: 12px !important;
  border: 1px solid rgba(255,255,255,0.18) !important;
  background: linear-gradient(180deg, rgba(0,230,255,0.25), rgba(0,230,255,0.08)) !important;
  color: #e8fbff !important;
  box-shadow: 0 0 18px rgba(0,230,255,0.35) !important;
}}

.atomic-app .q-btn-gold {{
  background: linear-gradient(180deg, rgba(255,200,87,0.3), rgba(255,200,87,0.06)) !important;
  color: #fff6dc !important;
  box-shadow: 0 0 18px rgba(255,200,87,0.35) !important;
}}

.atomic-app .q-input input,
.atomic-app .q-input textarea {{
  border-radius: 12px;
  border: 1px solid rgba(255,255,255,0.18);
  background: linear-gradient(180deg, rgba(0,230,255,0.25), rgba(0,230,255,0.08));
  color: #e8fbff;
  box-shadow: 0 0 18px rgba(0,230,255,0.35);
  padding: 10px 14px;
  height: 40px;
  line-height: 1.2;
  outline: none;
}}
.atomic-app .q-input input::placeholder,
.atomic-app .q-input textarea::placeholder {{
  color: rgba(232,251,255,0.75);
}}
.atomic-app .q-input input:focus,
.atomic-app .q-input textarea:focus {{
  border-color: rgba(0,230,255,0.55);
  box-shadow: 0 0 24px rgba(0,230,255,0.45);
}}
.atomic-app .q-input label,
.atomic-app .q-input .widget-label {{
  color: #e8fbff;
}}

.atomic-app .score {{
  font-size: 1.25rem;
  color: #00ffcc;
  text-shadow: 0 0 10px rgba(0,255,204,0.8);
  letter-spacing: 0.5px;
}}

/* Plot wrapper: expand content, no scrollbars */
.atomic-app .viewport {{
  border-radius: 16px;
  border: 1px solid rgba(255,255,255,0.18);
  overflow: visible; /* was hidden; let figures size naturally */
  background: rgba(0,0,0,0.35);
  box-shadow: 0 0 40px rgba(0, 230, 255, 0.18), inset 0 0 120px rgba(0,0,0,0.35);
  padding: 10px;
}}

/* Ensure ipywidgets Output areas don't clip or scroll */
.atomic-app .jupyter-widgets-output-area,
.atomic-app .output_subarea {{
  overflow: visible !important;
  max-height: none !important;
}}

/* Loading card elements (also used on the Loading screen) */
.loading-card {{
  width: 70%; max-width: 680px;
  background: rgba(12,12,30,0.95);
  border: 1px solid rgba(0,230,255,0.35);
  border-radius: 14px; padding: 18px 20px;
  box-shadow: 0 0 45px rgba(0,230,255,0.35), inset 0 0 18px rgba(255,255,255,0.05);
  text-align: center;
  color: #ffffff;
}}
.loading-title {{ font-size: 1.15rem; margin-bottom: 12px; letter-spacing: .3px; }}
.bar {{
  position: relative; height: 12px; width: 100%;
  background: rgba(255,255,255,0.08);
  border-radius: 999px; overflow: hidden;
  border: 1px solid rgba(255,255,255,0.15);
}}
.bar::before {{
  content: ""; position: absolute; top:0; left:-40%;
  height: 100%; width: 40%;
  background: linear-gradient(90deg, #00e6ff, #ff00cc);
  border-radius: 999px;
  animation: slide 1.2s linear infinite;
  box-shadow: 0 0 18px rgba(0,230,255,0.5);
}}
@keyframes slide {{ 0% {{ left: -40%; }} 100% {{ left: 100%; }} }}

/* ensure ipywidgets buttons pick up neon styles */
.atomic-app .widget-button.q-btn {{
  border-radius: 12px !important;
  border: 1px solid rgba(255,255,255,0.18) !important;
  background: linear-gradient(180deg, rgba(0,230,255,0.25), rgba(0,230,255,0.08)) !important;
  color: #e8fbff !important;
  box-shadow: 0 0 18px rgba(0,230,255,0.35) !important;
}}
.atomic-app .widget-button.q-btn-gold {{
  background: linear-gradient(180deg, rgba(255,200,87,0.3), rgba(255,200,87,0.06)) !important;
  color: #fff6dc !important;
  box-shadow: 0 0 18px rgba(255,200,87,0.35) !important;
}}

/* make slider description labels + readout numbers white */
.atomic-app .widget-label,
.atomic-app .widget-readout,
.atomic-app .widget-readout input {{
  color: #ffffff !important;
}}
</style>
"""
display(widgets.HTML(theme_css))

# ---------------------------
# Reusable bits
# ---------------------------
def pill(text, color):
    return f"<span style='display:inline-block;padding:2px 10px;border-radius:999px;border:1px solid rgba(255,255,255,0.2);background:rgba(255,255,255,0.05);color:{color};font-weight:700'>{text}</span>"

# ---------- Screen 1: Landing ----------
title = widgets.HTML(value="<h1 class='qtitle'>ATOMIC SHUTTLE</h1>")
subtitle = widgets.HTML(
    value="<div class='subtitle'>A game to learn how to transport an atom with a programmable trap.</div>"
)
atom_title = widgets.HTML(
    f"<div style='text-align:center;margin:8px 0 16px'><img src='{ATOM_URI}' width='90' height='90' style='filter:drop-shadow(0 0 8px rgba(255,255,255,0.8))'></div>"
)
landing_text = widgets.HTML(value=f"""
<div class="panel panel-strong lead">
  <b>Mission:</b> Move a single trapped <span style=\"color:#00e6ff\">atom</span> to the
      <span style=\"color:#ffc857\">Target Region</span> whithout losing it.<br/><br/>
      You have full control of the Trap’s Center {pill('μ(t): center', '#ff00cc')} and Trap's Half-Width {pill('σ(t): half-width', '#00e6ff')}.
      Beware! There might be some bumps in the road! Control the trap wisely to get the whole atom to the target zone.
</div>
""")

player_id = widgets.Text(
    value="",
    placeholder="Write your name",
    description="👤 Name:",
    layout=widgets.Layout(width="320px"),
)
player_id.style.description_width = "70px"
player_id.add_class("q-input")

start_btn = widgets.Button(description="Continue »", layout=widgets.Layout(width="160px", height="40px"))
start_btn.add_class("q-btn")

landing_controls = widgets.HBox(
    [player_id, start_btn],
    layout=widgets.Layout(justify_content="center", align_items="center")
)

landing_screen = widgets.VBox([title, subtitle, atom_title, landing_text, landing_controls],
                              layout=widgets.Layout(width="100%"))
landing_screen.add_class("screen")

# ---------- Screen 2: Overview / Instructions ----------
overview_header = widgets.HTML("<h1 class='qtitle'>How the Game Works</h1>")
overview_sub = widgets.HTML("<div class='subtitle'>Read this quick overview, then head to the Control Room.</div>")

what_is_it = widgets.HTML(value=f"""
<div class="panel panel-strong lead">
  <b>What is the game about?</b><br/>
  An atom is held in a programmable trap and we want to move the atom to the Target Region. We can shuttle the atom by changing the Trap's Center {pill('μ(t)', '#ff00cc')}
  and the Trap's Half-Width {pill('σ(t)', '#00e6ff')}. Sounds simple, but the atom can leave the trap if you don't control it correctly. Do you think that you can shuttle this atom to the Target Region without losing it?
</div>
""")

# Paths to pre-rendered assets used on the Overview page
MU_GIF      = "mu_demo.gif"
SIGMA_GIF   = "sigma_demo.gif"
EVOL_GIF    = "howitworks_final.gif"         # physics GIF (trap + |Ψ|²)
CURVES_PNG  = "howitworks_final_ui.png"      # static UI-style μ/σ plot
CONTOUR_PNG = "howitworks_final_contour.png" # static contour below

controls_demo_title = widgets.HTML("<div class='panel lead'><b>Controls, What They Do and Examples:</b> </div>")
mu_demo_title = widgets.HTML("<div class='panel lead'><b>Changing Trap's Center μ(t) without changing Trap's Half-Width </b> </div>")
mu_demo_text = widgets.HTML("""
<div class="panel lead">
  Changing the Trap's Center moves the trap's central position in space. This is the main mechanism for moving the atom to the Target Region.
  Be careful! If you move the Trap's Center too fast you can lose the atom, but if you move it too slowly it will never reach the Target Region.
</div>
""")
mu_gif_out = widgets.Output()

sigma_demo_title = widgets.HTML("<div class='panel lead'><b>Changing Trap's Half-Width σ(t) without changing Trap's Center</div>")
sigma_demo_text = widgets.HTML("""
<div class="panel lead">
  Just changing the Trap's Half-Width makes the trap either squeeze or expand.
  This changes how tightly the atom is confined or spread out. Squeeze the trap and you can get the atom excited, expand the trap and you can make it relax.
</div>
""")
sigma_gif_out = widgets.Output()

combo_title = widgets.HTML("<div class='panel lead'><b>Example: Changing both Trap's Center and Half-Width </b> </div>")
combo_text = widgets.HTML("""
<div class="panel lead">
  This is a simple example where we make the Trap's Center go forward while initially relax the Trap's Half-Width and then squeeze it.
</div>
""")
combo_plots_out   = widgets.Output()
combo_gif_out     = widgets.Output()
combo_contour_out = widgets.Output()

goal_panel = widgets.HTML("""
<div class="panel lead">
  <b>Goal:</b> Try to move the atom to the <span style="color:#ffc857">Target Region</span> by changing the Trap's Center and Trap's Half-Width.
  Do you think you are ready for the challenge? BEWARE! There might be some bumps along the way!
</div>
""")

to_controls_btn = widgets.Button(description="Enter Control Room »", icon="rocket", layout=widgets.Layout(width="220px"))
to_controls_btn.add_class("q-btn")
back_to_landing_btn = widgets.Button(description="« Back", layout=widgets.Layout(width="120px"))
back_to_landing_btn.add_class("q-btn")

overview_nav = widgets.HBox(
    [back_to_landing_btn, widgets.Box([], layout=widgets.Layout(flex="1 1 auto")), to_controls_btn],
    layout=widgets.Layout(align_items="center")
)

overview_screen = widgets.VBox([
    overview_header, overview_sub,
    what_is_it,
    controls_demo_title,
    mu_demo_title, mu_demo_text, widgets.HTML("<div class='panel viewport'>"), mu_gif_out, widgets.HTML("</div>"),
    sigma_demo_title, sigma_demo_text, widgets.HTML("<div class='panel viewport'>"), sigma_gif_out, widgets.HTML("</div>"),
    combo_title, combo_text,
    widgets.HBox([combo_plots_out, combo_gif_out], layout=widgets.Layout(gap="12px", align_items="flex-start")),
    widgets.HTML("<div class='panel viewport'>"),
    combo_contour_out,
    widgets.HTML("</div>"),
    goal_panel,
    overview_nav
], layout=widgets.Layout(width="100%"))
overview_screen.add_class("screen")

def _display_local(out, path, fmt, width=None, height=None, label=None):
    with out:
        clear_output(wait=True)
        if os.path.exists(path):
            with open(path, "rb") as f:
                img = widgets.Image(value=f.read(), format=fmt, width=width, height=height)
                display(img)
        else:
            label = label or path
            display(widgets.HTML(
                f"<div class='panel'>Missing asset <code>{label}</code>. "
                f"Run the pre-render script to generate it.</div>"
            ))

def load_overview_assets():
    _display_local(sigma_gif_out, SIGMA_GIF, "gif")
    _display_local(mu_gif_out,   MU_GIF,    "gif")
    _display_local(combo_plots_out,  CURVES_PNG,   "png")
    _display_local(combo_gif_out,    EVOL_GIF,     "gif")
    _display_local(combo_contour_out, CONTOUR_PNG, "png")

load_overview_assets()

# ---------- Screen 3: Controls ----------
help_text = widgets.HTML(
    value=(
        "<div class='panel'> Controls for the Trap's Center and Half-Width "
        f"Recall {pill('μ(t) Trap Center', '#ff00cc')} &nbsp; {pill('σ(t) Trap Half-Width', '#00e6ff')} "
        "→ Click <b>Engage Trap</b> button at the bottom of the page to run the simulation.</div>"
    )
)

# initial slider values
initial_points1 = -3.5 * np.ones(10)
initial_points2 =  1.0 * np.ones(10)

point_sliders1 = [
    widgets.FloatSlider(
        value=initial_points1[i], min=-9.0, max=9.0, step=0.01, continuous_update=True,
        description=fr'μ(t={i+1})', readout=True,
        layout=widgets.Layout(width="350px")
    ) for i in range(7)
]
for s in point_sliders1: s.style.handle_color = "#ff00cc"

point_sliders2 = [
    widgets.FloatSlider(
        value=initial_points2[i], min=0.1, max=5.0, step=0.01, continuous_update=True,
        description=fr'σ(t={i+1})', readout=True,
        layout=widgets.Layout(width="350px")
    ) for i in range(7)
]
for s in point_sliders2: s.style.handle_color = "#00e6ff"

mu_label    = widgets.HTML("<div style='color:#ff00cc;font-weight:600;margin:4px 0'>Define your Trap's Center μ</div>")
sigma_label = widgets.HTML("<div style='color:#00e6ff;font-weight:600;margin:12px 0 4px'>Define your Trap's Half-Width σ</div>")

sliders_left  = widgets.VBox([mu_label] + point_sliders1)
sliders_right = widgets.VBox([sigma_label] + point_sliders2)
sliders_area  = widgets.HBox([sliders_left, sliders_right],
                             layout=widgets.Layout(justify_content="space-between", align_items="flex-start"))

# SINGLE combined preview (replaces the two image previews)
plot_output1 = widgets.Output(layout=widgets.Layout(overflow='visible'))  # allow natural sizing

def V_Params_Smooth(mu_user, sigma_user, T_end):
    """Spline smoothing for UI-side preview."""
    t_vec = np.linspace(0, 20, 1000)
    N_t = len(t_vec); dt = t_vec[1] - t_vec[0]
    T_axis = np.arange(0, T_end + dt, T_end/7 , dtype=float)
    tck_mu = splrep(T_axis, mu_user, s=0)
    tck_sigma = splrep(T_axis, sigma_user, s=0)
    t_smooth = np.linspace(0, T_end, N_t)
    mu_smooth = BSpline(*tck_mu)(t_smooth)
    sigma_smooth = BSpline(*tck_sigma)(t_smooth)
    sigma_smooth = np.maximum(sigma_smooth, 0.1)
    return mu_smooth, sigma_smooth

# --- UI preview: smaller figure + white background so axes are visible
def update_controls_preview(point1_1, point2_1, point3_1, point4_1, point5_1, point6_1, point7_1,
                            point1_2, point2_2, point3_2, point4_2, point5_2, point6_2, point7_2):
    # control points (9 inc. initial)
    X = np.linspace(0, 8, 9)
    Y1 = np.array([-3.5, point1_1, point2_1, point3_1, point4_1, point5_1, point6_1, point7_1])
    Y2 = np.array([ 1.0,  point1_2, point2_2, point3_2, point4_2, point5_2, point6_2, point7_2])

    # smooth curves (still use the 20s backend horizon; we only *display* 0..8)
    Mu_Smooth, Sigma_Smooth = V_Params_Smooth(Y1, Y2, 20)
    # map to visible time [0,8]
    t_vis = np.linspace(0, 8, Mu_Smooth.size)

    L, alpha = 10, 5
    a, b = L - alpha, L
    mu_midline = 0.5*(a + b)

    with plot_output1:
        clear_output(wait=True)
        fig, ax = plt.subplots(1, 1, figsize=(6.0, 3.6), dpi=180, facecolor="white")
        ax.set_facecolor("white")

        # bands
        ax.axhspan(a, b, color='0.85', alpha=0.35, zorder=0)
        ax.axhline(mu_midline, ls='--', color='0.35', lw=1.2, zorder=1)
        ax.text(0.2, mu_midline + 0.3, 'Target region', fontsize=10, color='0.25')

        x_c, sigma_b = 3.0, 0.5
        ax.axhspan(x_c - 2*sigma_b, x_c + 2*sigma_b,
                   facecolor='tab:orange', alpha=0.15, edgecolor='tab:orange', lw=1.0)
        ax.axhline(x_c, ls='--', color='tab:orange', lw=1.2)
        ax.text(0.2, x_c + 0.3, 'Bump region', fontsize=10, color='tab:orange')

        # ribbon + curve
        ax.fill_between(t_vis, Mu_Smooth - 2*Sigma_Smooth, Mu_Smooth + 2*Sigma_Smooth,
                        color='tab:blue', alpha=0.18, linewidth=0, label=r'$\mu(t) \pm 2\sigma(t)$')
        ax.plot(t_vis, Mu_Smooth, color='tab:red', lw=2.2,
                path_effects=[pe.Stroke(linewidth=3.8, foreground='white'), pe.Normal()],
                label=r'$\mu(t)$')

        # control points
        ax.scatter(np.linspace(0, 8, len(Y1)), Y1, s=18, color='k', zorder=5)

        ax.set_xlim(0, 8); ax.set_ylim(-12, 12)
        ax.set_xlabel('Time'); ax.set_ylabel('Position')
        ax.set_title('Controls Graph of Traps Center and Width')
        ax.legend(loc='lower left', frameon=True, fontsize=9)
        plt.show()

interactive_plot = widgets.interactive(
    update_controls_preview,
    point1_1=point_sliders1[0],
    point2_1=point_sliders1[1],
    point3_1=point_sliders1[2],
    point4_1=point_sliders1[3],
    point5_1=point_sliders1[4],
    point6_1=point_sliders1[5],
    point7_1=point_sliders1[6],
    point1_2=point_sliders2[0],
    point2_2=point_sliders2[1],
    point3_2=point_sliders2[2],
    point4_2=point_sliders2[3],
    point5_2=point_sliders2[4],
    point6_2=point_sliders2[5],
    point7_2=point_sliders2[6]
)

# initial draw
update_controls_preview(
    *[s.value for s in point_sliders1],
    *[s.value for s in point_sliders2]
)

engage_btn = widgets.Button(description="⚡ Engage Trap", icon="play", layout=widgets.Layout(width="160px"))
engage_btn.add_class("q-btn")

to_overview_btn = widgets.Button(description="« Back", layout=widgets.Layout(width="120px"))
to_overview_btn.add_class("q-btn")

controls_toolbar = widgets.HBox(
    [to_overview_btn, widgets.Box([], layout=widgets.Layout(flex="1 1 auto")), engage_btn],
    layout=widgets.Layout(align_items="center")
)

controls_screen = widgets.VBox([
    widgets.HTML("<h1 class='qtitle'>Control Room</h1>"),
    widgets.HTML("<div class='subtitle'>Tune the trap schedule and run the experiment.</div>"),
    help_text,
    widgets.HTML("<div class='panel panel-strong'><b>Schedule</b></div>"),
    widgets.HTML("<div class='panel'>"),
    sliders_area,
    widgets.HTML("</div>"),
    widgets.HTML("<div class='panel panel-strong'><b>Preview</b></div>"),
    widgets.HTML("<div class='panel viewport'>"),
    plot_output1,
    widgets.HTML("</div>"),
    controls_toolbar
], layout=widgets.Layout(width="100%"))
controls_screen.add_class("screen")

# ---------- Screen 3.5: Loading Simulation ----------
loading_simulation_card = widgets.HTML(value="""
<div class="loading-card">
  <div class="loading-title">⚡ Solving Schrödinger…</div>
  <div class="bar"></div>
  <div style="margin-top:10px;opacity:.9">Loading your simulation — propagating the wavefunction.</div>
</div>
""")

loading_simulation_screen = widgets.VBox(
    [loading_simulation_card],
    layout=widgets.Layout(
        width='100%',
        min_height='688px',
        align_items='center',
        justify_content='center'
    )
)
loading_simulation_screen.add_class("screen")

# ---------- Screen 4: Results ----------
gif_output = widgets.Output(layout=widgets.Layout(overflow='visible'))  # allow full-size GIF + static fig
score_label = widgets.Label(value="Score: 0")
score_label.add_class("score")

congrats_html = widgets.HTML(value="""
<div class="panel panel-strong" style="text-align:center;font-size:1.1rem">
  <b>Simulation complete!</b> Here’s your trajectory and outcome.
</div>
""")

submit_button = widgets.Button(description="Submit to Scoreboard", icon="check", layout=widgets.Layout(width="210px"))
submit_button.add_class("q-btn")
play_again_button = widgets.Button(description="Play Again", icon="refresh", layout=widgets.Layout(width="150px"))
play_again_button.add_class("q-btn")

results_toolbar = widgets.HBox(
    [widgets.HTML("<div class='panel' style='padding:10px 14px'>"), score_label,
     widgets.Box([], layout=widgets.Layout(flex="1 1 auto")), submit_button, play_again_button,
     widgets.HTML("</div>")],
    layout=widgets.Layout(align_items="center")
)

results_screen = widgets.VBox([
    widgets.HTML("<h1 class='qtitle'>Results</h1>"),
    widgets.HTML("<div class='subtitle'>Wavefunction evolution and final probability in the target.</div>"),
    congrats_html,
    widgets.HTML("<div class='panel viewport'>"),
    gif_output,  # will contain the "Atoms Time Evolution" GIF and the "Final Plots for Score" figure
    widgets.HTML("</div>"),
    results_toolbar
], layout=widgets.Layout(width="100%"))
results_screen.add_class("screen")

qr_output = widgets.Output(layout=widgets.Layout(width="100%"))

# ---------- App container + simple router ----------
# Order: 0-Landing, 1-Overview, 2-Controls, 3-Loading, 4-Results, 5-QR
screen_stack = Stack(
    children=[landing_screen, overview_screen, controls_screen, loading_simulation_screen, results_screen, qr_output],
    layout=widgets.Layout(min_height='720px', width='100%')
)
screen_stack.selected_index = 0

app = widgets.VBox([screen_stack], layout=widgets.Layout(width='100%'))
app.add_class("atomic-app")
display(app)

def goto(idx: int):
    screen_stack.selected_index = idx

def backend():
    """
    Same GIF rendering as before, but the GIF's x-axis is mapped to 0..8 with
    integer ticks at 0,1,...,8 (to match the user input graph). Physics/scoring unchanged.
    """
    import io
    import numpy as np
    import matplotlib as mpl
    import matplotlib.pyplot as plt
    import matplotlib.colors as mcolors
    import matplotlib.patheffects as pe
    from matplotlib.ticker import FixedLocator, FixedFormatter
    from scipy.interpolate import splrep, BSpline
    from scipy.sparse import diags
    from scipy.sparse.linalg import eigsh
    from scipy.ndimage import gaussian_filter1d
    from PIL import Image as PILImage

    # ---------------- Time/space (physics timeline stays 0..20) ----------------
    t_vec = np.linspace(0, 20, 500); dt = t_vec[1] - t_vec[0]; N_t = len(t_vec)
    # CHANGED: visualization timeline explicitly 0..8 over the same N_t samples
    t_vis = np.linspace(0, 8, N_t)

    r_vec = np.linspace(-40, 40, 2500); npts = len(r_vec)
    dx = r_vec[1] - r_vec[0]

    # ---------------- FFT helpers ----------------
    def get_k_grid(npts, xx):
        k = np.fft.fftfreq(npts)
        dx_loc = (max(xx) - min(xx)) / npts
        return 2*np.pi * k / dx_loc
    def fft_ortho(f):  return 1.0/np.sqrt(np.size(f)) * np.fft.fft(f)
    def ifft_ortho(f): return np.sqrt(np.size(f)) * np.fft.ifft(f)

    grid_k = get_k_grid(npts, r_vec)
    T_k = 0.5 * grid_k**2

    # ---------------- User controls smoothing ----------------
    def V_Params_Smooth_Internal(mu_user, sigma_user, N_t_local, T_end):
        # keep physics knots as-is (unchanged)
        T_axis = np.linspace(0, T_end, 8)  # 8 intervals → 9 pts in the original design
        tck_mu = splrep(T_axis, mu_user, s=0)
        tck_sigma = splrep(T_axis, sigma_user, s=0)
        t_smooth = np.linspace(0, T_end, N_t_local)
        mu_smooth = BSpline(*tck_mu)(t_smooth)
        sigma_smooth = np.maximum(BSpline(*tck_sigma)(t_smooth), 0.1)
        return mu_smooth, sigma_smooth

    mu0, sigma0 = -3.5, 1.0
    mu_user    = [mu0]    + [slider.value for slider in point_sliders1]  # original 8 knots
    sigma_user = [sigma0] + [slider.value for slider in point_sliders2]  # original 8 knots
    Mu, Sigma  = V_Params_Smooth_Internal(mu_user, sigma_user, N_t, t_vec[-1])

    # CHANGED: build "UI knot" copies with a duplicated final value so we can
    # place control dots exactly at integer ticks 0..8 on the GIF.
    mu_knots    = mu_user + [mu_user[-1]]
    sigma_knots = sigma_user + [sigma_user[-1]]

    # ---------------- Potentials ----------------
    V_dynamic = np.zeros((npts, N_t), dtype=float)
    V_dynamic[:, 0] = -np.exp(-(r_vec - mu0)**2 / (2*sigma0**2))
    for s in range(1, N_t-1):
        V_dynamic[:, s] = -np.exp(-(r_vec - Mu[s])**2 / (2*Sigma[s]**2))
    V_dynamic[:, -1] = V_dynamic[:, -2]

    # Absorbers (visible window)
    boundary_left, boundary_right = -15, 15
    boundary_scale = 0.001
    Vabs = np.zeros_like(r_vec)
    Vabs[r_vec > boundary_right] = boundary_scale*(np.cosh(r_vec[r_vec > boundary_right]-boundary_right)-1)
    Vabs[r_vec < boundary_left]  = boundary_scale*(np.cosh(-(r_vec[r_vec < boundary_left]-boundary_left))-1)

    # Obstacle (bump)
    V_rightbump = 0.2*np.exp(-(r_vec - 3)**2/(2*0.5**2))
    V_bumps = V_rightbump

    # Target/score region
    L, alpha = 10, 5
    a, b = L - alpha, L
    mu_mid = 0.5*(a + b)

    # ---------------- Sparse ground-state solver ----------------
    def TISE(x, dx_local, V):
        N = len(x)
        off  = (-0.5/dx_local**2) * np.ones(N-1)
        main = (1.0/dx_local**2)  * np.ones(N)
        H = diags([off, main, off], offsets=[-1, 0, 1], format='csc') + diags(V, offsets=0, format='csc')
        vals, vecs = eigsh(H, k=1, which='SA')
        return vals, vecs

    # Initial and reference states
    _, U0   = TISE(r_vec, dx, -np.exp(-(r_vec - mu0   )**2 / (2*sigma0**2)))
    _, Uref = TISE(r_vec, dx, -np.exp(-(r_vec - mu_mid)**2 / (2*sigma0**2)))
    phi_ref = Uref[:, 0] / np.sqrt(np.sum(np.abs(Uref[:, 0])**2) * dx)
    density_ref = np.abs(phi_ref)**2

    # ---------------- Propagate + capture (unchanged) ----------------
    f_r = (U0[:, 0] / np.sqrt(np.sum(np.abs(U0[:, 0])**2) * dx)).astype(complex)

    x_ds = 3
    max_frames = 80
    frame_step = max(1, N_t // max_frames)
    capture_idx = np.arange(0, N_t, frame_step, dtype=int)
    if capture_idx[-1] != N_t - 1:
        capture_idx = np.append(capture_idx, N_t - 1)

    r_small = r_vec[::x_ds]
    dens_small = np.zeros((r_small.size, capture_idx.size), dtype=float)

    cap_j = 0
    for n in range(N_t):
        V_t = V_dynamic[:, n] + V_bumps - 1j*Vabs
        f_r = np.exp(-1j*V_t*dt/2) * f_r
        f_k = fft_ortho(f_r)
        f_k = np.exp((-1j*T_k)*dt) * f_k
        f_r = ifft_ortho(f_k)
        f_r = np.exp(-1j*V_t*dt/2) * f_r
        if cap_j < capture_idx.size and n == capture_idx[cap_j]:
            dens_small[:, cap_j] = np.abs(f_r[::x_ds])**2
            cap_j += 1

    dens_img = gaussian_filter1d(dens_small, sigma=0.9, axis=0, mode='nearest')
    dens_img = gaussian_filter1d(dens_img,  sigma=0.6, axis=1, mode='nearest')

    # ---------------- Score ----------------
    density_final = np.abs(f_r)**2
    overlap = np.vdot(density_ref, density_final) * dx
    overlap_norm = np.vdot(density_ref, density_ref) * dx + 1e-18
    Score = float((overlap / overlap_norm) * 100.0)
    Score = min(100.0, Score)
    score_label.value = f"Score: {Score:.2f}"

    # ---------------- GIF (map x to 0..8; ticks at integers 0..8) ----------------
    frames = []
    with mpl.rc_context({'figure.constrained_layout.use': False}):
        fig, ax = plt.subplots(figsize=(7.2, 2.9), dpi=170)
        fig.subplots_adjust(left=0.11, right=0.88, bottom=0.20, top=0.86)

        norm0 = mcolors.PowerNorm(gamma=0.6, vmin=0.0, vmax=max(1e-12, dens_img.max()))
        cax = fig.add_axes([0.90, 0.22, 0.02, 0.62])
        sm = mpl.cm.ScalarMappable(norm=norm0, cmap='Greys'); sm.set_array([])
        cb = fig.colorbar(sm, cax=cax)  # cb.set_label(r'$|\Psi(x,t)|^2$')

        # ticks & labels to match user graph (0..8)
        integer_ticks = list(range(0, 9))
        integer_labels = [str(i) for i in integer_ticks]
        ctrl_x = np.linspace(0, 8, len(mu_knots))  # 9 dots exactly at 0..8

        for k in range(1, dens_img.shape[1] + 1):
            ax.clear(); fig.axes[:] = [ax, cax]

            # extent uses visualization time (0..8)
            t_left  = 0.0
            t_right = max(1e-6, t_vis[capture_idx[k-1]])

            ax.imshow(
                dens_img[:, :k],
                origin='lower',
                aspect='auto',
                extent=[t_left, t_right, r_vec[0], r_vec[-1]],
                cmap='Greys',
                norm=norm0,
                interpolation='lanczos'
            )

            # target region
            ax.axhspan(a, b, facecolor='0.6', alpha=0.25, linewidth=0)
            ax.axhline(0.5*(a+b), ls='--', color='0.35', lw=1.0)
            ax.text(0.35, b + 0.3, 'Target region', fontsize=9, color='0.35')

            # bump region
            w = np.real(V_rightbump).clip(min=0); wsum = w.sum()
            x_c = np.sum(r_vec * w) / wsum
            sigma_b = np.sqrt(np.sum(w * (r_vec - x_c) ** 2) / wsum)
            ax.axhspan(x_c - 2*sigma_b, x_c + 2*sigma_b, facecolor='tab:orange', alpha=0.10, linewidth=0)
            ax.axhline(x_c, color='tab:orange', ls='--', lw=1.0)
            ax.text(0.35, x_c + 0.3, 'Bump region', fontsize=9, color='tab:orange')

            # μ(t) ribbon + line → plotted vs t_vis
            t_now = t_vis[capture_idx[:k]]
            ax.fill_between(t_now, Mu[capture_idx[:k]] - 2*Sigma[capture_idx[:k]],
                            Mu[capture_idx[:k]] + 2*Sigma[capture_idx[:k]],
                            color='tab:blue', alpha=0.16, linewidth=0)
            ax.plot(t_now, Mu[capture_idx[:k]], color='crimson', lw=2.0,
                    path_effects=[pe.Stroke(linewidth=3.4, foreground='white'), pe.Normal()])

            # control-point dots at integer ticks 0..8
            ax.scatter(ctrl_x, mu_knots, s=18, color='k', zorder=10)

            ax.set_xlim(0, 8)
            ax.set_ylim(boundary_left, boundary_right)
            ax.set_xlabel('Time', fontsize=12, color='black')
            ax.set_ylabel('Position', fontsize=12, color='black')
            ax.set_title('Atoms Time Evolution', fontsize=14, color='black')

            ax.xaxis.set_major_locator(FixedLocator(integer_ticks))
            ax.xaxis.set_major_formatter(FixedFormatter(integer_labels))

            ax.tick_params(colors='black')
            for spine in ax.spines.values(): spine.set_color('black')

            buf = io.BytesIO()
            fig.savefig(buf, format='png', facecolor='white', bbox_inches=None, pad_inches=0.04)
            buf.seek(0)
            pil = PILImage.open(buf).convert('RGB')
            w_target = 900
            scale = w_target / pil.width
            pil = pil.resize((w_target, max(1, int(pil.height * scale))), resample=PILImage.LANCZOS)
            frames.append(pil.convert('P', palette=0))
            buf.close()
        plt.close(fig)

    gif_path = os.path.join(TMPDIR, "Dynamic_WF_and_Pot.gif")
    if frames:
        frames[0].save(
            gif_path,
            save_all=True,
            append_images=frames[1:],
            duration=90,
            loop=0,
            optimize=True
        )

    # ---------------- Display GIF + static final plot (unchanged) ----------------
    with gif_output:
        clear_output(wait=True)
        with open(gif_path, "rb") as f:
            display(widgets.Image(value=f.read(), format='gif'))

        with mpl.rc_context({'figure.constrained_layout.use': False}):
            fig_ref, ax_ref = plt.subplots(figsize=(5.6, 3.0), dpi=190)
            fig_ref.subplots_adjust(left=0.12, right=0.97, bottom=0.24, top=0.86)
            dens_ref   = np.abs(phi_ref)**2
            dens_final = np.abs(f_r)**2
            ax_ref.axvspan(a, b, color='0.6', alpha=0.15, linewidth=0, label='Target Region')
            ax_ref.plot(r_vec, dens_final, lw=1.6, color='k', label='Users Result')
            ax_ref.plot(r_vec, dens_ref,  lw=1.6, color='crimson', ls='--', label='Objective')
            ax_ref.set_xlim(-15, 15)
            ax_ref.set_xlabel('Position', fontsize=12, color='black')
            ax_ref.set_ylabel(r'$|\Psi(x,t_f)|^2$', fontsize=12, color='black')
            ax_ref.set_title(f'Score: {Score:.2f}%', fontsize=12, color='black')
            ax_ref.tick_params(colors='black')
            for spine in ax_ref.spines.values(): spine.set_color('black')
            ax_ref.legend(loc='upper left', frameon=True, fontsize=9)
            fig_ref.patch.set_facecolor('white'); ax_ref.set_facecolor('white')
            display(fig_ref); plt.close(fig_ref)

    return Score

# ---------- Screen actions ----------
def enter_overview(_):
    if not player_id.value.strip():
        player_id.value = "Player"
    goto(1)
    load_overview_assets()

def back_to_landing(_):
    goto(0)

def to_controls(_):
    goto(2)
    update_controls_preview(
        *[s.value for s in point_sliders1],
        *[s.value for s in point_sliders2],
    )

def run_experiment(_):
    global score, is_running
    if is_running:
        return  # prevent double trigger
    is_running = True
    try:
        # Disable controls while running
        engage_btn.disabled = True
        to_overview_btn.disabled = True
        for s in point_sliders1 + point_sliders2:
            s.disabled = True

        # Navigate to dedicated loading page
        goto(3)  # loading_simulation_screen

        # Run simulation (blocking)
        score = backend()

        # Show results page
        goto(4)  # results_screen
    finally:
        # Re-enable controls
        engage_btn.disabled = False
        to_overview_btn.disabled = False
        for s in point_sliders1 + point_sliders2:
            s.disabled = False
        is_running = False

def play_again(_):
    goto(2)

def submit_score(_):
    user_name_final = (player_id.value or "Player").replace(' ', '-')
    Stringbuilder = (
        "https://gmscoreboard.com/api/set-score/"
        "?tagid=a12af85e0e5057f1762cad57c65f58bf"
        f"&player={user_name_final}&score={int(score)}"
    )
    qr = qrcode.make(Stringbuilder)
    qr_path = os.path.join(TMPDIR, "qr_code.png")
    qr.save(qr_path)
    with qr_output:
        clear_output(wait=True)
        display(Image(qr_path))
    goto(5)  # QR page

# Wire buttons
start_btn.on_click(enter_overview)
back_to_landing_btn.on_click(back_to_landing)
to_controls_btn.on_click(to_controls)
to_overview_btn.on_click(lambda _: goto(1))
engage_btn.on_click(run_experiment)
play_again_button.on_click(play_again)
submit_button.on_click(submit_score)


HTML(value='\n<style>\n.atomic-app {\n  position: relative;\n  background-image: url(\'data:image/svg+xml;base…